In [17]:
import re
from collections import Counter

In [18]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [19]:
# create a frequency table of all the words of the document
all_words = Counter(words(open('spell_correction_document.txt').read()))

In [20]:
all_words

Counter({'the': 79809,
         'project': 288,
         'gutenberg': 263,
         'ebook': 87,
         'of': 40024,
         'adventures': 17,
         'sherlock': 101,
         'holmes': 467,
         'by': 6735,
         'sir': 177,
         'arthur': 34,
         'conan': 4,
         'doyle': 5,
         '15': 47,
         'in': 22023,
         'our': 1065,
         'series': 128,
         'copyright': 51,
         'laws': 233,
         'are': 3630,
         'changing': 43,
         'all': 4143,
         'over': 1282,
         'world': 362,
         'be': 6155,
         'sure': 123,
         'to': 28765,
         'check': 38,
         'for': 6941,
         'your': 1279,
         'country': 423,
         'before': 1362,
         'downloading': 5,
         'or': 5352,
         'redistributing': 7,
         'this': 4063,
         'any': 1203,
         'other': 1501,
         'header': 7,
         'should': 1297,
         'first': 1174,
         'thing': 303,
         'seen': 444,
  

In [21]:
# check frequency of a random word, say, 'chair'
all_words['chair']

135

In [22]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [23]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

You just saw two functions. The edits_one() function and the edits_two() function. The edits_one() function creates all the possible words that are one edit distance away from the input word. Most of the words that this function creates are garbage, i.e. they are not valid English words. For example, if you pass the word ‘laern’ (misspelling of the word ‘learn’) to edits_one(), it will create a list where the word ‘lgern’ will be present since it is an edit away from the word ‘laern’. But it’s not an English word. Only a subset of the words will be actual English words.

In [24]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))


Similarly, the edits_two() function creates a list of all the possible words that are two edits away from the input word. Most of these words will also be garbage.

In [25]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

Known function -  It will remove all those words which are not present in "Spell Correction Document"

The known() function filters out the valid English word from a list of given words. It uses the frequency distribution as a dictionary that was created using the seed document. If the words created using edits_one() and edits_two() are not in the dictionary, they’re discarded.

In [26]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

Now, the function possible_corrections() returns a list of all the potential words that can be the correct alternative spelling. For example, let’s say the user has typed the word ‘wut’ which is wrong. There are multiple words that could be the correct spelling of this word such as ‘cut’, ‘but’, ‘gut’, etc. This functions will return all these words for the given incorrect word ‘wut’. But, how does this function find all these word suggestions exactly? It works as follows:

It first checks if the word is correct or not, i.e. if the word typed by the user is a present in the dictionary or not. If the word is present, it returns no spelling suggestions since it is already a correct dictionary word.

If the user types a word which is not a dictionary word, then it creates a list of all the known words that are one edit distance away. If there are no valid words in the list created by edits_one() only then this function fetches a list of all known words that are two edits away from the input word

If there are no known words that are two edits away, then the function returns the  original input word. This means that there are no alternatives that the spell corrector could find. Hence, it simply returns the original word.

 

Finally, there is the prob() function. The function returns the probability of an input word. This is exactly why you need a seed document instead of a dictionary. A dictionary only contains a list of all correct English words. But, a seed document not only contains all the correct words but it could also be used to create a frequency distribution of all these words. This frequency will be taken into consideration when there are more than one possibly correct words for a given misspelled word. Let’s say the user has input the word ‘wut’. The correct alternative to this word could be one of these words - ‘cut’, ‘but’ and ‘gut’, etc. The possible_corrections() function will return all these words. But the prob() function will create a probability associated with each of these suggestions and return the one with highest probability. Suppose, if a word ‘but’ is present 2000 times out of a total of million words in the seed document, then it’s probability would be 2000/1000000, i.e. 0.002.

In [27]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [28]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

336
{'mmonney', 'donney', 'conney', 'imonney', 'omonney', 'monneye', 'monnei', 'pmonney', 'monneay', 'molney', 'zmonney', 'monneyc', 'momnney', 'moneey', 'monneyl', 'xmonney', 'mlonney', 'bmonney', 'mosnney', 'mkonney', 'mowney', 'monneyy', 'mondney', 'mofnney', 'mionney', 'moniney', 'monnem', 'monhey', 'moxnney', 'monneyp', 'mponney', 'monnsy', 'monneys', 'monneyd', 'monneiy', 'monnpey', 'mronney', 'motnney', 'monnevy', 'monneyt', 'monneny', 'moinney', 'moqnney', 'monjey', 'monneyu', 'mdnney', 'monwey', 'qonney', 'monnepy', 'monnwy', 'monnsey', 'monnuey', 'ponney', 'montney', 'monneby', 'monnye', 'moniey', 'monnew', 'mcnney', 'monnexy', 'monniy', 'mogney', 'monniey', 'monnly', 'monoey', 'monaey', 'mononey', 'monneqy', 'moanney', 'monnec', 'monnen', 'konney', 'monny', 'monnee', 'mouney', 'monnry', 'monngy', 'monndey', 'mzonney', 'monneyv', 'jonney', 'monneyr', 'vonney', 'monnxy', 'nmonney', 'monnehy', 'monsney', 'monneo', 'minney', 'movnney', 'monnay', 'monneky', 'mhnney', 'monneu', 'm

In [29]:
print(known(edits_one("monney")))

{'money', 'monkey'}


In [30]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'money', 'monkey'}


In [31]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'money', 'monkey'}


In [32]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0002922233626303688
5.378344097491451e-06


In [33]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [34]:
# test spell check
print(spell_check("monney"))

Did you mean money?


In [36]:
print(len(edits_one("emfasize")))

442


In [40]:
len(set(edits_two("emfasize")))

90902

In [41]:
print(possible_corrections("emfasize"))

{'emphasize'}
